# CycleGAN


Ref: CHU, Casey; ZHMOGINOV, Andrey; SANDLER, Mark. Cyclegan, a master of steganography. arXiv preprint arXiv:1712.02950, 2017.

In [26]:
# load libraries
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os

In [27]:
# from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dropout, Concatenate, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Model
from keras.optimizers import Adam
from keras.initializers import RandomNormal
from keras.layers.merge import add

In [28]:
import tensorflow as tf
from keras.engine.topology import Layer
from keras.engine import InputSpec

class ReflectionPadding2D(Layer):
    """ ref. https://stackoverflow.com/questions/50677544/reflection-padding-conv2d
    """
    def __init__(self, padding=(1, 1), **kwargs):
        self.padding = tuple(padding)
        self.input_spec = [InputSpec(ndim=4)]
        super(ReflectionPadding2D, self).__init__(**kwargs)

    def compute_output_shape(self, s):
        """ If you are using "channels_last" configuration"""
        return (s[0], s[1] + 2 * self.padding[0], s[2] + 2 * self.padding[1], s[3])

    def call(self, x, mask=None):
        w_pad,h_pad = self.padding
        return tf.pad(x, [[0,0], [h_pad,h_pad], [w_pad,w_pad], [0,0] ], 'REFLECT')

## Data loader

In [29]:
from data_loader2 import DataLoader

## Helper function for the generator

In [30]:
def conv7s1(layer_input, filters, final, weight_init):
    y = ReflectionPadding2D(padding =(3,3))(layer_input)
    y = Conv2D(filters, kernel_size=(7,7), strides=1, padding='valid', kernel_initializer = weight_init)(y)
    if final:
        y = Activation('tanh')(y)
    else:
        y = InstanceNormalization(axis = -1, center = False, scale = False)(y)
        y = Activation('relu')(y)

    return y

def downsample(layer_input,filters, weight_init):
    y = Conv2D(filters, kernel_size=(3,3), strides=2, padding='same', kernel_initializer = weight_init)(layer_input)
    y = InstanceNormalization(axis = -1, center = False, scale = False)(y)
    y = Activation('relu')(y)
    
    return y

def residual(layer_input, filters, weight_init):
    shortcut = layer_input
    y = ReflectionPadding2D(padding =(1,1))(layer_input)
    y = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='valid', kernel_initializer = weight_init)(y)
    y = InstanceNormalization(axis = -1, center = False, scale = False)(y)
    y = Activation('relu')(y)

    y = ReflectionPadding2D(padding =(1,1))(y)
    y = Conv2D(filters, kernel_size=(3, 3), strides=1, padding='valid', kernel_initializer = weight_init)(y)
    y = InstanceNormalization(axis = -1, center = False, scale = False)(y)

    return add([shortcut, y])

def upsample(layer_input,filters, weight_init):
    y = Conv2DTranspose(filters, kernel_size=(3, 3), strides=2, padding='same', kernel_initializer = weight_init)(layer_input)
    y = InstanceNormalization(axis = -1, center = False, scale = False)(y)
    y = Activation('relu')(y)
    
    return y

## Generator

In [31]:
def build_generator(img_shape, gf, weight_init):
    """ U-Net Generator
        :param gf: generator filter size in the first layer
        
        
        ref.: Kaiming He et al., “Deep Residual Learning for Image Recognition” 
              10 December 2015, https://arxiv.org/abs/1512.03385
    """
    # Image input
    img = Input(shape=img_shape)

    y = img

    y = conv7s1(y, gf, False, weight_init)
    y = downsample(y, gf * 2, weight_init)
    y = downsample(y, gf * 4, weight_init)
    y = residual(y, gf * 4, weight_init)
    y = residual(y, gf * 4, weight_init)
    y = residual(y, gf * 4, weight_init)
    y = residual(y, gf * 4, weight_init)
    y = residual(y, gf * 4, weight_init)
    y = residual(y, gf * 4, weight_init)
    y = residual(y, gf * 4, weight_init)
    y = residual(y, gf * 4, weight_init)
    y = residual(y, gf * 4, weight_init)
    y = upsample(y, gf * 2, weight_init)
    y = upsample(y, gf, weight_init)
    y = conv7s1(y, 3, True, weight_init)
    output = y

    return Model(img, output)

## Helper function for Discriminator

In [32]:
def conv4(layer_input, filters, weight_init, stride = 2, norm=True):
    y = Conv2D(filters, kernel_size=(4,4), strides=stride, padding='same', kernel_initializer = weight_init)(layer_input)
    if norm:
        y = InstanceNormalization(axis = -1, center = False, scale = False)(y)
    y = LeakyReLU(0.2)(y)

    return y

# Discriminator

In [33]:
def build_discriminator(img_shape, df, weight_init):

    img = Input(shape=img_shape)

    y = conv4(img, df, weight_init, stride = 2, norm = False)
    y = conv4(y, df * 2, weight_init, stride = 2)
    y = conv4(y, df * 4, weight_init, stride = 4)
    y = conv4(y, df * 8, weight_init, stride = 1)

    output = Conv2D(1, kernel_size=4, strides=1, padding='same', kernel_initializer = weight_init)(y)

    return Model(img, output)

In [34]:
def sample_images(epoch, batch_i,
                  g_AB, g_BA, # generators
                  dataset_name, data_loader):
    os.makedirs('images/%s' % dataset_name, exist_ok=True)
    r, c = 2, 3 # rows, columns

    imgs_A = data_loader.load_data(domain="A", batch_size=1, is_testing=True)
    imgs_B = data_loader.load_data(domain="B", batch_size=1, is_testing=True)

    # Translate images to the other domain
    fake_B = g_AB.predict(imgs_A)
    fake_A = g_BA.predict(imgs_B)
    # Translate back to original domain
    reconstr_A = g_BA.predict(fake_B)
    reconstr_B = g_AB.predict(fake_A)

    gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    titles = ['Original', 'Translated', 'Reconstructed']
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):  # rows
        for j in range(c):  # columns
            axs[i,j].imshow(gen_imgs[cnt])
            axs[i, j].set_title(titles[j])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%s/%d_%d.png" % (dataset_name, epoch, batch_i))
    plt.close()

## Perform training of the GAN

In [35]:
def train(epochs, 
          g_AB, g_BA, # generators
          d_A, d_B, # discriminators
          combined, # full GAN
          dataset_name, img_rows, img_cols,
          batch_size=1, sample_interval=50,
          ):
    
    # Calculate output shape of D (PatchGAN)
    patch = int(img_rows / 2**4)
    disc_patch = (patch, patch, 1)

    # class that loads the data in batches
    data_loader = DataLoader(dataset_name=dataset_name,
                             img_res=(img_rows, img_cols))

    start_time = datetime.datetime.now()

    # Adversarial loss ground truths
    valid = np.ones((batch_size,) + disc_patch)
    fake = np.zeros((batch_size,) + disc_patch)

    for epoch in range(epochs):
        for batch_i, (imgs_A, imgs_B) in enumerate(data_loader.load_batch(batch_size)):

            # ----------------------
            #  Train Discriminators
            # ----------------------

            # Translate images to opposite domain
            fake_B = g_AB.predict(imgs_A)
            fake_A = g_BA.predict(imgs_B)

            # Train the discriminators (original images = real / translated = Fake)
            dA_loss_real = d_A.train_on_batch(imgs_A, valid)
            dA_loss_fake = d_A.train_on_batch(fake_A, fake)
            dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

            dB_loss_real = d_B.train_on_batch(imgs_B, valid)
            dB_loss_fake = d_B.train_on_batch(fake_B, fake)
            dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

            # Total disciminator loss
            d_loss = 0.5 * np.add(dA_loss, dB_loss)


            # ------------------
            #  Train Generators
            # ------------------

            # Train the generators
            g_loss = combined.train_on_batch([imgs_A, imgs_B],
                                             [valid, valid,
                                              imgs_A, imgs_B,
                                              imgs_A, imgs_B])

            elapsed_time = datetime.datetime.now() - start_time

            # Plot the progress
            print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                    % ( epoch, epochs,
                                                                        batch_i, data_loader.n_batches,
                                                                        d_loss[0], 100*d_loss[1],
                                                                        g_loss[0],
                                                                        np.mean(g_loss[1:3]),
                                                                        np.mean(g_loss[3:5]),
                                                                        np.mean(g_loss[5:6]),
                                                                        elapsed_time))

            # If at save interval => save generated image samples
            if batch_i % sample_interval == 0:
                sample_images(epoch, batch_i, 
                              g_AB, g_BA, # generators
                              dataset_name, data_loader)

## main()

In [36]:
img_rows = 128
img_cols = 128
channels = 3
img_shape = (img_rows, img_cols, channels)

# Configure data loader
dataset_name = 'apple2orange'

In [37]:
weight_init = RandomNormal(mean=0., stddev=0.02)

In [38]:
# Loss weights
lambda_cycle = 10.0               # Cycle-consistency loss
lambda_id = 0.1 * lambda_cycle    # Identity loss

In [39]:
# optimizer
optimizer = Adam(0.0002, 0.5)

In [40]:
# Number of filters in the first layer of G and D
gf = 32
df = 64

In [41]:
# Build and compile the discriminators
d_A = build_discriminator(img_shape, df, weight_init)
d_B = build_discriminator(img_shape, df, weight_init)
d_A.compile(loss='mse',
    optimizer=optimizer,
    metrics=['accuracy'])
d_B.compile(loss='mse',
    optimizer=optimizer,
    metrics=['accuracy'])

d_A.summary()

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 64, 64, 64)        3136      
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 32, 32, 128)       131200    
_________________________________________________________________
instance_normalization_59 (I (None, 32, 32, 128)       0         
_________________________________________________________________
leaky_re_lu_18 (LeakyReLU)   (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 8, 8, 256)         5245

In [42]:
#-------------------------
# Construct Computational
#   Graph of Generators
#-------------------------

# Build the generators
g_AB = build_generator(img_shape, gf, weight_init)
g_BA = build_generator(img_shape, gf, weight_init)

In [43]:
# Input images from both domains
img_A = Input(shape=img_shape)
img_B = Input(shape=img_shape)

In [44]:
# Translate images to the other domain
fake_B = g_AB(img_A)
fake_A = g_BA(img_B)

In [45]:
# Translate images back to original domain
reconstr_A = g_BA(fake_B)
reconstr_B = g_AB(fake_A)

In [46]:
# Identity mapping of images
img_A_id = g_BA(img_A)
img_B_id = g_AB(img_B)

In [47]:
# For the combined model we will only train the generators
d_A.trainable = False
d_B.trainable = False

# Discriminators determines validity of translated images
valid_A = d_A(fake_A)
valid_B = d_B(fake_B)

In [48]:
# Combined model trains generators to fool discriminators
combined = Model(inputs=[img_A, img_B],
                      outputs=[ valid_A, valid_B,
                                reconstr_A, reconstr_B,
                                img_A_id, img_B_id ])
combined.compile(loss=['mse', 'mse',
                            'mae', 'mae',
                            'mae', 'mae'],
                    loss_weights=[  1, 1,
                                    lambda_cycle, lambda_cycle,
                                    lambda_id, lambda_id ],
                    optimizer=optimizer)

In [49]:
epochs = 10 # 200
train(epochs=epochs, batch_size=1, sample_interval=200,
      g_AB=g_AB, g_BA=g_BA, d_A=d_A, d_B=d_B, combined=combined,
      dataset_name=dataset_name, img_rows=img_rows, img_cols=img_cols
      )

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[Epoch 0/2] [Batch 0/995] [D loss: 2.000792, acc:  28%] [G loss: 29.565210, adv: 5.295656, recon: 0.864134, id: 0.956028] time: 0:00:38.102565 
[Epoch 0/2] [Batch 1/995] [D loss: 4.242106, acc:  25%] [G loss: 22.354902, adv: 2.932665, recon: 0.749814, id: 0.863008] time: 0:00:40.991532 
[Epoch 0/2] [Batch 2/995] [D loss: 3.104193, acc:  25%] [G loss: 19.969934, adv: 3.053705, recon: 0.629672, id: 0.812977] time: 0:00:43.269533 
[Epoch 0/2] [Batch 3/995] [D loss: 3.100769, acc:  26%] [G loss: 21.144304, adv: 3.783855, recon: 0.617508, id: 0.729241] time: 0:00:45.609533 
[Epoch 0/2] [Batch 4/995] [D loss: 4.996716, acc:  20%] [G loss: 20.225487, adv: 3.791628, recon: 0.575086, id: 0.587869] time: 0:00:48.011532 
[Epoch 0/2] [Batch 5/995] [D loss: 2.492306, acc:  25%] [G loss: 19.631672, adv: 2.330153, recon: 0.677704, id: 0.785379] time: 0:00:50.436533 
[Epoch 0/2] [Batch 6/995] [D loss: 3.546727, acc:  24%] [G loss: 15.401932, adv: 2.195482, recon: 0.500669, id: 0.578063] time: 0:00:52.

[Epoch 0/2] [Batch 57/995] [D loss: 0.580573, acc:  48%] [G loss: 13.380840, adv: 0.654946, recon: 0.546718, id: 0.508078] time: 0:03:29.930535 
[Epoch 0/2] [Batch 58/995] [D loss: 0.659199, acc:  39%] [G loss: 12.706997, adv: 0.539571, recon: 0.525923, id: 0.381460] time: 0:03:33.382532 
[Epoch 0/2] [Batch 59/995] [D loss: 0.692901, acc:  40%] [G loss: 14.649514, adv: 0.488370, recon: 0.619799, id: 0.626680] time: 0:03:36.874532 
[Epoch 0/2] [Batch 60/995] [D loss: 0.687771, acc:  39%] [G loss: 9.404800, adv: 0.677464, recon: 0.363756, id: 0.372094] time: 0:03:40.271532 
[Epoch 0/2] [Batch 61/995] [D loss: 0.807573, acc:  39%] [G loss: 9.734346, adv: 0.772186, recon: 0.370794, id: 0.372183] time: 0:03:43.628533 
[Epoch 0/2] [Batch 62/995] [D loss: 0.926175, acc:  39%] [G loss: 9.457861, adv: 0.506066, recon: 0.381674, id: 0.479276] time: 0:03:47.032535 
[Epoch 0/2] [Batch 63/995] [D loss: 0.622785, acc:  41%] [G loss: 10.122426, adv: 0.717768, recon: 0.394109, id: 0.338855] time: 0:03

[Epoch 0/2] [Batch 114/995] [D loss: 0.469201, acc:  43%] [G loss: 12.652173, adv: 0.519788, recon: 0.530389, id: 0.466921] time: 0:06:47.981971 
[Epoch 0/2] [Batch 115/995] [D loss: 0.437379, acc:  47%] [G loss: 8.706905, adv: 0.556209, recon: 0.347183, id: 0.287023] time: 0:06:51.218971 
[Epoch 0/2] [Batch 116/995] [D loss: 0.409992, acc:  44%] [G loss: 8.449896, adv: 0.530428, recon: 0.338182, id: 0.382568] time: 0:06:54.457972 
[Epoch 0/2] [Batch 117/995] [D loss: 0.522760, acc:  38%] [G loss: 11.172731, adv: 0.450359, recon: 0.465789, id: 0.604241] time: 0:06:57.731973 
[Epoch 0/2] [Batch 118/995] [D loss: 0.464435, acc:  44%] [G loss: 13.524414, adv: 0.545192, recon: 0.573117, id: 0.218311] time: 0:07:00.969971 
[Epoch 0/2] [Batch 119/995] [D loss: 0.439891, acc:  49%] [G loss: 9.757846, adv: 0.670568, recon: 0.381017, id: 0.261948] time: 0:07:04.188974 
[Epoch 0/2] [Batch 120/995] [D loss: 0.370526, acc:  53%] [G loss: 6.914371, adv: 0.618993, recon: 0.259891, id: 0.288274] time

[Epoch 0/2] [Batch 171/995] [D loss: 0.611803, acc:  44%] [G loss: 9.734123, adv: 0.590919, recon: 0.391458, id: 0.315656] time: 0:10:16.333991 
[Epoch 0/2] [Batch 172/995] [D loss: 0.585086, acc:  36%] [G loss: 7.929626, adv: 0.477401, recon: 0.319132, id: 0.307868] time: 0:10:19.952992 
[Epoch 0/2] [Batch 173/995] [D loss: 0.546076, acc:  35%] [G loss: 8.393938, adv: 0.535476, recon: 0.335740, id: 0.295675] time: 0:10:23.583993 
[Epoch 0/2] [Batch 174/995] [D loss: 0.472532, acc:  47%] [G loss: 7.022552, adv: 0.491349, recon: 0.275166, id: 0.266643] time: 0:10:27.425662 
[Epoch 0/2] [Batch 175/995] [D loss: 0.306571, acc:  58%] [G loss: 8.964760, adv: 0.648291, recon: 0.348630, id: 0.367933] time: 0:10:31.748999 
[Epoch 0/2] [Batch 176/995] [D loss: 0.392669, acc:  48%] [G loss: 8.277041, adv: 0.508000, recon: 0.332517, id: 0.274991] time: 0:10:36.220009 
[Epoch 0/2] [Batch 177/995] [D loss: 0.405051, acc:  43%] [G loss: 6.428337, adv: 0.417937, recon: 0.253414, id: 0.274069] time: 0

...

[Epoch 1/2] [Batch 719/995] [D loss: 0.311779, acc:  49%] [G loss: 5.645506, adv: 0.632848, recon: 0.200532, id: 0.181249] time: 1:48:08.396739 
[Epoch 1/2] [Batch 720/995] [D loss: 0.242371, acc:  58%] [G loss: 4.934506, adv: 0.631213, recon: 0.165816, id: 0.121976] time: 1:48:12.468739 
[Epoch 1/2] [Batch 721/995] [D loss: 0.344141, acc:  33%] [G loss: 8.092752, adv: 0.402087, recon: 0.331360, id: 0.203819] time: 1:48:16.580414 
[Epoch 1/2] [Batch 722/995] [D loss: 0.283451, acc:  57%] [G loss: 7.518183, adv: 0.758247, recon: 0.276052, id: 0.196887] time: 1:48:20.332414 
[Epoch 1/2] [Batch 723/995] [D loss: 0.282048, acc:  55%] [G loss: 7.453466, adv: 0.554286, recon: 0.293389, id: 0.284469] time: 1:48:24.388414 
[Epoch 1/2] [Batch 724/995] [D loss: 0.296432, acc:  50%] [G loss: 6.029949, adv: 0.531625, recon: 0.226196, id: 0.152077] time: 1:48:28.314448 
[Epoch 1/2] [Batch 725/995] [D loss: 0.359645, acc:  38%] [G loss: 8.959525, adv: 0.642438, recon: 0.349533, id: 0.208141] time: 1

[Epoch 1/2] [Batch 776/995] [D loss: 0.228056, acc:  67%] [G loss: 8.388510, adv: 0.529808, recon: 0.343908, id: 0.258264] time: 1:51:45.374811 
[Epoch 1/2] [Batch 777/995] [D loss: 0.342643, acc:  39%] [G loss: 5.467175, adv: 0.582197, recon: 0.197677, id: 0.180254] time: 1:51:49.098813 
[Epoch 1/2] [Batch 778/995] [D loss: 0.289998, acc:  53%] [G loss: 6.051492, adv: 0.591072, recon: 0.219262, id: 0.253306] time: 1:51:52.932812 
[Epoch 1/2] [Batch 779/995] [D loss: 0.170778, acc:  73%] [G loss: 6.317460, adv: 0.692412, recon: 0.218814, id: 0.382014] time: 1:51:56.628811 
[Epoch 1/2] [Batch 780/995] [D loss: 0.332696, acc:  51%] [G loss: 5.686399, adv: 0.674349, recon: 0.199112, id: 0.161123] time: 1:52:00.242813 
[Epoch 1/2] [Batch 781/995] [D loss: 0.353519, acc:  39%] [G loss: 5.829674, adv: 0.543278, recon: 0.216770, id: 0.201021] time: 1:52:03.883813 
[Epoch 1/2] [Batch 782/995] [D loss: 0.257555, acc:  52%] [G loss: 8.553956, adv: 0.539658, recon: 0.348106, id: 0.341296] time: 1

[Epoch 1/2] [Batch 833/995] [D loss: 0.256554, acc:  61%] [G loss: 7.692868, adv: 0.550570, recon: 0.302531, id: 0.262087] time: 1:55:14.393060 
[Epoch 1/2] [Batch 834/995] [D loss: 0.103077, acc:  88%] [G loss: 4.667461, adv: 0.791514, recon: 0.140433, id: 0.095588] time: 1:55:18.176060 
[Epoch 1/2] [Batch 835/995] [D loss: 0.259799, acc:  57%] [G loss: 8.011917, adv: 0.520044, recon: 0.319166, id: 0.288121] time: 1:55:22.243062 
[Epoch 1/2] [Batch 836/995] [D loss: 0.252886, acc:  65%] [G loss: 5.178819, adv: 0.624333, recon: 0.176437, id: 0.209822] time: 1:55:26.418061 
[Epoch 1/2] [Batch 837/995] [D loss: 0.257007, acc:  55%] [G loss: 4.392170, adv: 0.529302, recon: 0.150658, id: 0.154032] time: 1:55:30.383060 
[Epoch 1/2] [Batch 838/995] [D loss: 0.174387, acc:  76%] [G loss: 6.847048, adv: 0.706934, recon: 0.246898, id: 0.194928] time: 1:55:34.343060 
[Epoch 1/2] [Batch 839/995] [D loss: 0.403163, acc:  37%] [G loss: 5.787187, adv: 0.520195, recon: 0.217534, id: 0.221338] time: 1

[Epoch 1/2] [Batch 890/995] [D loss: 0.508831, acc:  17%] [G loss: 5.922586, adv: 0.416290, recon: 0.234701, id: 0.191547] time: 1:58:45.608175 
[Epoch 1/2] [Batch 891/995] [D loss: 0.249528, acc:  59%] [G loss: 5.706783, adv: 0.649373, recon: 0.200313, id: 0.074345] time: 1:58:49.316176 
[Epoch 1/2] [Batch 892/995] [D loss: 0.071683, acc:  95%] [G loss: 6.190675, adv: 0.810961, recon: 0.210053, id: 0.132478] time: 1:58:52.681176 
[Epoch 1/2] [Batch 893/995] [D loss: 0.205381, acc:  70%] [G loss: 4.607935, adv: 0.660959, recon: 0.148512, id: 0.147645] time: 1:58:55.981175 
[Epoch 1/2] [Batch 894/995] [D loss: 0.354240, acc:  44%] [G loss: 4.817256, adv: 0.478089, recon: 0.176268, id: 0.168034] time: 1:58:59.404712 
[Epoch 1/2] [Batch 895/995] [D loss: 0.557843, acc:  39%] [G loss: 4.333529, adv: 0.308708, recon: 0.167954, id: 0.216415] time: 1:59:03.002712 
[Epoch 1/2] [Batch 896/995] [D loss: 0.191229, acc:  69%] [G loss: 4.528046, adv: 0.723398, recon: 0.141248, id: 0.106424] time: 1

[Epoch 1/2] [Batch 947/995] [D loss: 0.457698, acc:  26%] [G loss: 3.627424, adv: 0.352389, recon: 0.133269, id: 0.116492] time: 2:02:19.691912 
[Epoch 1/2] [Batch 948/995] [D loss: 0.229053, acc:  61%] [G loss: 5.398139, adv: 0.676970, recon: 0.185339, id: 0.195105] time: 2:02:23.510429 
[Epoch 1/2] [Batch 949/995] [D loss: 0.260605, acc:  54%] [G loss: 5.585316, adv: 0.625794, recon: 0.198109, id: 0.186265] time: 2:02:27.396430 
[Epoch 1/2] [Batch 950/995] [D loss: 0.209306, acc:  66%] [G loss: 7.585356, adv: 0.660257, recon: 0.284256, id: 0.273688] time: 2:02:31.208428 
[Epoch 1/2] [Batch 951/995] [D loss: 0.334059, acc:  55%] [G loss: 5.509734, adv: 0.555353, recon: 0.203099, id: 0.181245] time: 2:02:35.055429 
[Epoch 1/2] [Batch 952/995] [D loss: 0.273138, acc:  53%] [G loss: 4.670329, adv: 0.594194, recon: 0.157573, id: 0.150484] time: 2:02:39.029431 
[Epoch 1/2] [Batch 953/995] [D loss: 0.401825, acc:  30%] [G loss: 6.341588, adv: 0.419822, recon: 0.257006, id: 0.149194] time: 2